In [3]:
! for n in 0 1 2 3; do ssh slave$n uptime; done

 09:16:50 up 1 day, 10:50,  load average: 0.10, 0.05, 0.01
 09:16:50 up 1 day, 10:50,  load average: 0.10, 0.05, 0.01
 09:16:51 up 1 day, 10:50,  load average: 0.10, 0.05, 0.01
 09:16:51 up 1 day, 10:50,  load average: 0.10, 0.05, 0.01
